In [1]:
from google.cloud import bigquery
import pandas as pd 
import chime   
import seaborn as sns
client = bigquery.Client("gcp-wow-rwds-ai-checkout-dev")

/opt/homebrew/lib/python3.11/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

In [151]:
check = client.query(
"""
CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.adobe_carto_catalogue_homepage_ciid_search_term_test`  AS (
    with 
    
    cartology_campaign_asset_impressions_in_adobe AS (
        SELECT 
            date_time,
            shopper_id_map.shopper_id, 
            crn_map.crn, 
            post_evar6 AS page_name,
            site_platform,
            
            CASE WHEN SPLIT(post_evar6,":")[SAFE_OFFSET(2)] = 'catalogue' OR LOWER(post_evar7) = 'catalogue' THEN 1 ELSE 0 END AS catalogue_flag,    
            CASE WHEN post_evar6 = 'ww-sm:homepage' THEN 1 ELSE 0 END AS homepage_flag, 
            CASE WHEN post_mvvar2 IS NOT NULL THEN SPLIT(post_mvvar2,":")[SAFE_OFFSET(0)] ELSE "Not Found" END AS booking_id,
            CASE WHEN post_mvvar2 IS NOT NULL THEN SPLIT(post_mvvar2,":")[SAFE_OFFSET(3)] ELSE NULL END AS ci_id,
            post_evar130,
            post_evar51 AS search_term1,
            post_evar52 AS search_term2,
            post_evar53 AS search_term3,
            LTRIM(RTRIM(REGEXP_EXTRACT(CASE 
                WHEN site_platform = "Web" THEN lower(post_evar51)
                WHEN site_platform = "App" THEN lower(post_evar199)
            END, r"^([^|]*)"))) AS search_term_edit
            
        FROM `gcp-wow-food-wlx-digaspt-dev.prod_adobe_data.grs_adobe` adobe

        -- shopper id
        LEFT JOIN `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_visitorid_shopperid_map_window` shopper_id_map
            ON adobe.prop9 = shopper_id_map.visitor_id
            AND adobe.date_time BETWEEN shopper_id_map.effective_time AND shopper_id_map.expiry_time
        
        -- crn
        LEFT JOIN `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_visitorid_crn_map_window` crn_map
            ON adobe.prop9 = crn_map.visitor_id
            AND adobe.date_time BETWEEN crn_map.effective_time and crn_map.expiry_time

            

        WHERE 
            site_name = "Supermarkets"
        AND 
            DATE(date_time) >= DATE("2023-10-01")
        AND    
            DATE(date_time) <= DATE("2024-01-01") --CURRENT_DATE("Australia/Sydney")
        /*
        AND         
            (
                -- Catalogue
                (SPLIT(post_evar6,":")[SAFE_OFFSET(2)] = 'catalogue' OR LOWER(post_evar7) = 'catalogue')
                OR 
                -- Homepage
                (post_evar6 = 'ww-sm:homepage') 
                OR 
                -- ci_id 
                (SPLIT(post_mvvar2,":")[SAFE_OFFSET(3)] IS NOT NULL)
            )
        */
        AND      
            post_channel NOT LIKE "Order Confirmation Section"
        AND 
            (shopper_id_map.shopper_id IS NOT NULL OR crn_map.crn IS NOT NULL)
            
        --AND SPLIT(post_mvvar2,":")[SAFE_OFFSET(0)] = "WOW20005374"
    
    ) 

    SELECT * FROM cartology_campaign_asset_impressions_in_adobe 

);
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

""


In [ ]:
check = client.query(
"""

DECLARE campaigns_to_be_analyzed_array ARRAY<STRING>;
DECLARE start_index INT64 DEFAULT 0;
DECLARE end_index INT64; 
DECLARE current_campaign STRING;
DECLARE campaign_run_start_time DATETIME;
DECLARE query_start_time DATETIME;
DECLARE query_end_time DATETIME;

/*
SET campaigns_to_be_analyzed_array = (
    SELECT ARRAY_AGG(DISTINCT booking_id) 
    FROM `gcp-wow-cart-data-dev-d4d7.davide.adobe_carto_catalogue_homepage_ciid_search_term_test` events
    LEFT JOIN `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` cc 
        ON events.booking_id = cc.booking_number
    WHERE booking_id <> "WOW20005374"
    AND booking_id NOT IN (SELECT DISTINCT booking_id FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_nine_final_results`)
    AND cc.media_type IS NOT NULL
    AND LOWER(cc.media_type) NOT LIKE "%search%"
);
*/ 


SET campaigns_to_be_analyzed_array = (
        with step_one AS (
            SELECT DISTINCT 
                booking_number,
                media_type,
                CASE WHEN media_type <> "Display Aisle and Category" THEN 1 ELSE 0 END AS non_display_aisle_asset
            FROM `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` 
            ORDER BY booking_number
        ), 
        step_two AS (
            SELECT cc.* EXCEPT(individual_product_string), ARRAY_AGG(DISTINCT individual_product_string IGNORE NULLS ORDER BY individual_product_string) AS skus
            FROM `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` cc 
            LEFT JOIN (SELECT DISTINCT booking_number FROM step_one WHERE non_display_aisle_asset = 1) non
                ON non.booking_number = cc.booking_number 
            WHERE non.booking_number IS NULL
        )
        SELECT ARRAY_AGG(DISTINCT booking_number) FROM step_two WHERE booking_number <> "WOW20002352" AND ARRAY_LENGTH(skus) >= 6
);


SET end_index = ARRAY_LENGTH(campaigns_to_be_analyzed_array);



LOOP
    IF start_index >= end_index THEN 
        LEAVE;
    END IF;

SET current_campaign = campaigns_to_be_analyzed_array[OFFSET(start_index)];



SET campaign_run_start_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');
SET query_start_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');
DELETE FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_one_events` WHERE booking_id = current_campaign;
INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_one_events`
--CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_one_events` AS 
    SELECT 
        a.*, 
        cc.media_type, 
        cc.campaign_start_date, 
        cc.campaign_end_date, 
        cc.media_start_date, 
        cc.media_end_date, 
        dim.quoteline_sku, 
        dim.media_type AS original_media_type
    FROM `gcp-wow-cart-data-dev-d4d7.davide.adobe_carto_catalogue_homepage_ciid_search_term_test` a
    LEFT JOIN `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` cc
        ON a.booking_id = cc.booking_number 
        AND a.ci_id = cc.line_name
    LEFT JOIN `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns` dim
        ON dim.booking_number = a.booking_id 
        AND dim.line_name = a.ci_id
    WHERE a.booking_id = current_campaign
    AND cc.line_name IS NOT NULL
;

SET query_end_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');

    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3`
        SELECT  
            current_campaign AS campaign_id,
            "1" AS query_step,
            "Adobe Web Events" AS query_type,
            query_start_time, 
            query_end_time, 
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) AS query_duration_in_seconds,
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) / 60 AS query_duration_in_minutes
    ;

SET query_start_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');
DELETE FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_two_dates` WHERE booking_id = current_campaign;
INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_two_dates` 
--CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_two_dates` AS 
    SELECT DISTINCT 
        booking_id,
        campaign_start_date,
        campaign_end_date 
    FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_one_events`
    WHERE booking_id = current_campaign 
;

SET query_end_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');

    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3`
        SELECT  
            current_campaign AS campaign_id,
            "2" AS query_step,
            "Campaign Dates" AS query_type,
            query_start_time, 
            query_end_time, 
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) AS query_duration_in_seconds,
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) / 60 AS query_duration_in_minutes
    ;

SET query_start_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');
/*
DELETE FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_three_page_names` WHERE booking_id = current_campaign;
INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_three_page_names`
--CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_three_page_names` AS 
    SELECT 
        booking_id, 
        campaign_start_date,
        campaign_end_date,
        media_start_date,
        media_end_date,
        site_platform,
        media_type,
        ci_id, 
        search_term3,
        page_name,
        post_evar130,
        ARRAY_AGG(DISTINCT sku ORDER BY sku) AS skus,
        ARRAY_AGG(DISTINCT am.ArticleDescription ORDER BY am.ArticleDescription) AS descriptions
    FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_one_events`,
    UNNEST(SPLIT(quoteline_sku, ",")) AS sku
    LEFT JOIN `gcp-wow-ent-im-wowx-cust-prod.adp_wowx_dm_masterdata_view.dim_article_master_v` am 
        ON am.Article = sku 
    WHERE 
    post_evar130 = "impression"
    AND booking_id = current_campaign
    AND am.SalesOrg = 1005
    GROUP BY 1,2,3,4,5,6,7,8,9,10,11
;
*/

DELETE FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_three_page_names_2` WHERE booking_id = current_campaign;
INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_three_page_names_2`
--CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_three_page_names_2` AS 
    SELECT  
        booking_id, 
        DATE(date_time) AS impression_date,
        campaign_start_date,
        campaign_end_date,
        media_start_date,
        media_end_date,
        site_platform,
        media_type,
        ci_id, 
        page_name,
        ARRAY_AGG(DISTINCT sku ORDER BY sku) AS skus,
        ARRAY_AGG(DISTINCT am.ArticleDescription ORDER BY am.ArticleDescription) AS descriptions
    FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_one_events`,
    UNNEST(SPLIT(quoteline_sku, ",")) AS sku
    LEFT JOIN `gcp-wow-ent-im-wowx-cust-prod.adp_wowx_dm_masterdata_view.dim_article_master_v` am 
        ON am.Article = sku 
    WHERE 
        post_evar130 = "impression"
    AND 
        booking_id = current_campaign
    AND 
        am.SalesOrg = 1005
    GROUP BY 1,2,3,4,5,6,7,8,9,10
;

SET query_end_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');

    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3`
        SELECT  
            current_campaign AS campaign_id,
            "3" AS query_step,
            "Campaign Pages" AS query_type,
            query_start_time, 
            query_end_time, 
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) AS query_duration_in_seconds,
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) / 60 AS query_duration_in_minutes
    ;

SET query_start_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');
DELETE FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_four_exposed_shoppers` WHERE booking_id = current_campaign;
INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_four_exposed_shoppers`
--CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_four_exposed_shoppers` AS 
    SELECT DISTINCT 
        events.booking_id, 
        page_names.campaign_start_date,
        page_names.campaign_end_date,
        page_names.media_start_date,
        page_names.media_end_date,
        events.crn,
        events.date_time AS exposure_time,
        page_names.media_type,
        ARRAY_TO_STRING(skus, ",") AS skus
    FROM `gcp-wow-cart-data-dev-d4d7.davide.adobe_carto_catalogue_homepage_ciid_search_term_test` events
    LEFT JOIN `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_three_page_names_2` page_names
        ON page_names.booking_id = events.booking_id
        AND page_names.ci_id = events.ci_id
    WHERE events.booking_id = current_campaign
    AND page_names.ci_id IS NOT NULL
    AND events.crn IS NOT NULL
;

SET query_end_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');

    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3`
        SELECT  
            current_campaign AS campaign_id,
            "4" AS query_step,
            "Exposed Shoppers" AS query_type,
            query_start_time, 
            query_end_time, 
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) AS query_duration_in_seconds,
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) / 60 AS query_duration_in_minutes
    ;

SET query_start_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');
DELETE FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_five_non_exposed_shoppers` WHERE booking_id = current_campaign;
INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_five_non_exposed_shoppers`
--CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_five_non_exposed_shoppers` AS 
    SELECT DISTINCT 
        dates.booking_id,
        events.crn,
        events.date_time AS non_exposure_time,
        media_type
    FROM `gcp-wow-cart-data-dev-d4d7.davide.adobe_carto_catalogue_homepage_ciid_search_term_test` events 
    CROSS JOIN (SELECT DISTINCT booking_id, campaign_start_date, campaign_end_date FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_two_dates` WHERE booking_id = current_campaign) dates
    INNER JOIN (SELECT DISTINCT page_name, impression_date, media_type FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_three_page_names_2`) page_names
        ON events.page_name = page_names.page_name 
        AND DATE(events.date_time) = page_names.impression_date
    LEFT JOIN (SELECT DISTINCT crn FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_four_exposed_shoppers` WHERE booking_id = current_campaign) exposed_shoppers 
        ON exposed_shoppers.crn = events.crn 
    WHERE exposed_shoppers.crn IS NULL
    AND DATE(events.date_time) >= dates.campaign_start_date 
    AND DATE(events.date_time) <= dates.campaign_end_date 
    AND events.crn IS NOT NULL
;

SET query_end_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');

    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3`
        SELECT  
            current_campaign AS campaign_id,
            "5" AS query_step,
            "Non-Exposed Shoppers" AS query_type,
            query_start_time, 
            query_end_time, 
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) AS query_duration_in_seconds,
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) / 60 AS query_duration_in_minutes
    ;

SET query_start_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');
DELETE FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_six_exposed_campaign_purchases` WHERE booking_id = current_campaign;
INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_six_exposed_campaign_purchases`
--CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_six_exposed_campaign_purchases` AS 

        -- Exposed Shopper Transactions
        SELECT 
            DATE(exposure.exposure_time) AS impression_date, 
            exposure.booking_id, 
            media_type,
            "Exposed" AS exposure_context,
            exposure.crn AS exposure_crn,
            transactions.order_context,
            transactions.shopper_identification_number AS converted_shopper_identification_number,
            transactions.start_txn_time AS start_txn_time,
            transactions.basket_key AS basket_key,
            transactions.article AS article,
            transactions.tot_net_incld_gst AS tot_net_incld_gst,
            CASE WHEN transactions.start_txn_time < DATETIME_ADD(exposure.exposure_time, INTERVAL 3 DAY) 
                THEN "3 Days" 
                ELSE "7 Days" 
            END AS exposure_to_purchase_window
        
        FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_four_exposed_shoppers` exposure,
        UNNEST(SPLIT(skus, ",")) AS individual_campaign_product_string

        -- WHere crn is not null
        LEFT JOIN `gcp-wow-cart-data-dev-d4d7.davide.test_transactions` transactions 
            ON exposure.crn = transactions.shopper_identification_number
            AND individual_campaign_product_string = transactions.article 
            AND 
                (
                    (
                        transactions.start_txn_time >= exposure.exposure_time
                        AND 
                        transactions.start_txn_time <= DATETIME_ADD(exposure.exposure_time, INTERVAL 7 DAY)
                    )
                )
            AND transactions.shopper_identification_number IS NOT NULL
            AND booking_id = current_campaign
;
SET query_end_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');

    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3`
        SELECT  
            current_campaign AS campaign_id,
            "6" AS query_step,
            "Exposed Transactions" AS query_type,
            query_start_time, 
            query_end_time, 
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) AS query_duration_in_seconds,
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) / 60 AS query_duration_in_minutes
    ;

SET query_start_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');
DELETE FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_seven_non_exposed_campaign_purchases` WHERE booking_id = current_campaign;
INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_seven_non_exposed_campaign_purchases`
--CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_seven_non_exposed_campaign_purchases` AS 
    -- Exposed Shopper Transactions
        SELECT 
            DATE(exposure.non_exposure_time) AS impression_date, 
            exposure.booking_id, 
            media_type,
            "Not Exposed" AS exposure_context,
            exposure.crn AS exposure_crn,
            transactions.order_context,
            transactions.shopper_identification_number AS converted_shopper_identification_number,
            transactions.start_txn_time AS start_txn_time,
            transactions.basket_key AS basket_key,
            transactions.article AS article,
            transactions.tot_net_incld_gst AS tot_net_incld_gst,
            CASE WHEN transactions.start_txn_time < DATETIME_ADD(exposure.non_exposure_time, INTERVAL 3 DAY) THEN "3 Days" ELSE "7 Days" END AS exposure_to_purchase_window
        
        FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_five_non_exposed_shoppers` exposure 
        CROSS JOIN (SELECT DISTINCT booking_id, ARRAY_TO_STRING(skus, ",") skus FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_three_page_names_2` WHERE booking_id = current_campaign),
        UNNEST(SPLIT(skus, ",")) AS individual_campaign_product_string

        -- WHere crn is not null
        LEFT JOIN `gcp-wow-cart-data-dev-d4d7.davide.test_transactions` transactions 
            ON exposure.crn = transactions.shopper_identification_number
            AND individual_campaign_product_string = transactions.article 
            AND 
                (
                    (
                        transactions.start_txn_time >= exposure.non_exposure_time
                        AND 
                        transactions.start_txn_time <= DATETIME_ADD(exposure.non_exposure_time, INTERVAL 7 DAY)
                    )
                )
            AND transactions.shopper_identification_number IS NOT NULL
        AND exposure.booking_id = current_campaign
;
SET query_end_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');

    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3`
        SELECT  
            current_campaign AS campaign_id,
            "7" AS query_step,
            "Non-Exposed Transactions" AS query_type,
            query_start_time, 
            query_end_time, 
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) AS query_duration_in_seconds,
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) / 60 AS query_duration_in_minutes
    ;

SET query_start_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');
DELETE FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_eight_distinct_transactions`  WHERE booking_id = current_campaign;
INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_eight_distinct_transactions` 
--CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_eight_distinct_transactions` AS 
    SELECT  
        booking_id,
        exposure_context,
        exposure_crn,
        converted_shopper_identification_number AS converted_crn, 
        order_context,
        basket_key, 
        article, 
        tot_net_incld_gst,
        ARRAY_AGG(DISTINCT media_type IGNORE NULLS ORDER BY media_type) AS media_types
    FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_six_exposed_campaign_purchases` 
    WHERE booking_id = current_campaign
    GROUP BY 1,2,3,4,5,6,7,8

    UNION ALL

    SELECT  
        booking_id,
        exposure_context,
        exposure_crn,
        converted_shopper_identification_number AS converted_crn, 
        order_context,
        basket_key, 
        article, 
        tot_net_incld_gst,
        ARRAY_AGG(DISTINCT media_type IGNORE NULLS ORDER BY media_type) AS media_types
    FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_seven_non_exposed_campaign_purchases` 
    WHERE booking_id = current_campaign
    GROUP BY 1,2,3,4,5,6,7,8
;
SET query_end_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');

    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3`
        SELECT  
            current_campaign AS campaign_id,
            "8" AS query_step,
            "Distinct Conversions" AS query_type,
            query_start_time, 
            query_end_time, 
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) AS query_duration_in_seconds,
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) / 60 AS query_duration_in_minutes
    ;

SET query_start_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');
DELETE FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_nine_final_results` WHERE booking_id = current_campaign;
INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_nine_final_results`
--CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_nine_final_results` AS 
    SELECT 
        booking_id,
        ARRAY_TO_STRING(media_types, ",") AS media_types,
        exposure_context, 
        COUNT(DISTINCT exposure_crn) AS visitors,
        COUNT(DISTINCT converted_crn) AS buyers,
        SAFE_DIVIDE(COUNT(DISTINCT converted_crn) , COUNT(DISTINCT exposure_crn)) AS conversion_rate,
        SAFE_DIVIDE(COUNT(DISTINCT basket_key) , COUNT(DISTINCT converted_crn)) AS baskets_per_purchased_shopper, 
        SAFE_DIVIDE(COUNT(DISTINCT basket_key) , COUNT(DISTINCT converted_crn)) AS baskets_per_visited_shopper,
        SAFE_DIVIDE(SUM(tot_net_incld_gst) , COUNT(DISTINCT exposure_crn)) AS spend_per_visited_shopper,
        SAFE_DIVIDE(SUM(tot_net_incld_gst) , COUNT(DISTINCT converted_crn)) AS spend_per_purchased_shopper 
    FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_eight_distinct_transactions`  
    WHERE booking_id = current_campaign
    GROUP BY 1,2,3
;
SET query_end_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');

    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3`
        SELECT  
            current_campaign AS campaign_id,
            "9" AS query_step,
            "Final Results" AS query_type,
            query_start_time, 
            query_end_time, 
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) AS query_duration_in_seconds,
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) / 60 AS query_duration_in_minutes
    ;

SET query_end_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');

    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3`
        SELECT  
            current_campaign AS campaign_id,
            "10" AS query_step,
            "Total Runtime" AS query_type,
            query_start_time, 
            query_end_time, 
            DATETIME_DIFF(query_end_time, campaign_run_start_time, SECOND) AS query_duration_in_seconds,
            DATETIME_DIFF(query_end_time, campaign_run_start_time, SECOND) / 60 AS query_duration_in_minutes
    ;

    
    SET start_index = start_index + 1;
END LOOP;

"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

In [142]:
check = client.query(
"""
--SELECT * FROM  `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_nine_final_results`
--SELECT * FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_seven_non_exposed_campaign_purchases` LIMIT 5
--SELECT * FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_five_non_exposed_shoppers` LIMIT 4;
CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_five_non_exposed_shoppers` AS 
    SELECT 
        booking_id, 
        s5.*
    FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_five_non_exposed_shoppers` s5
    CROSS JOIN (SELECT DISTINCT booking_id FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_nine_final_results`);
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

""


In [158]:
check = client.query(
"""
    SELECT  
        udo_page_name,
        udo_page_type,
        udo_app_section,
        udo_acs_stitch_id,
        SPLIT(udo_acs_stitch_id, ":")[OFFSET(2)] AS tealium_media_type,
        MIN(DATE(post_time, "Australia/Sydney")) AS earliest_visit,
        MAX(DATE(post_time, "Australia/Sydney")) AS latest_visit,
        COUNT(DISTINCT visitor_id) AS n_visitors,
        COUNT(DISTINCT CASE WHEN SPLIT(udo_acs_stitch_id, ":")[OFFSET(0)] = dim_campaign.booking_number AND SPLIT(udo_acs_stitch_id, ":")[OFFSET(3)] = dim_campaign.line_name THEN visitor_id END) AS n_visitors_exposed,
        COUNT(DISTINCT CASE WHEN SPLIT(udo_acs_stitch_id, ":")[OFFSET(0)] <> dim_campaign.booking_number AND SPLIT(udo_acs_stitch_id, ":")[OFFSET(3)] <> dim_campaign.line_name THEN visitor_id END) AS n_visitors_not_exposed
    FROM `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_events` all_events
    LEFT JOIN `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns` dim_campaign 
        ON TRIM(SPLIT(udo_acs_stitch_id, ":")[OFFSET(0)]) = TRIM(dim_campaign.booking_number)
        AND TRIM(SPLIT(udo_acs_stitch_id, ":")[OFFSET(3)]) = TRIM(dim_campaign.line_name)
    WHERE 
        1 = 1 
    -- AND udo_acs_stitch_id IS NOT NULL 
    -- AND udo_page_name LIKE "%crackers%" "ww-sm:shop:browse:snacks-confectionery:biscuits-crackers:crackers"
    AND DATE(post_time, "Australia/Sydney") >= DATE("2023-12-01")
    AND DATE(post_time, "Australia/Sydney") < DATE("2024-01-01")
    AND SPLIT(udo_acs_stitch_id, ":")[OFFSET(0)] = "WOW20005811"
    -- AND udo_tealium_datasource = "qurl3t"
    -- AND udo_app_section IS NOT NULL
    -- AND SPLIT(udo_acs_stitch_id, ":")[OFFSET(5)] = "app" 
    -- AND udo_acs_stitch_id NOT LIKE "%search%"
    GROUP BY 1,2,3,4,5


"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,udo_page_name,udo_page_type,udo_app_section,udo_acs_stitch_id,tealium_media_type,earliest_visit,latest_visit,n_visitors,n_visitors_exposed,n_visitors_not_exposed
0,ww-sm:shop:browse:beauty-personal-care:dental-care:toothpaste,productcategory,None,WOW20005811:health-and-beauty---category-banner:banner-category:CI-10077697:0048219:web,banner-category,2023-12-21,2023-12-26,27,27,0
1,ww-sm:shop:browse:beauty-personal-care:period-care:pads-pants,productcategory,None,WOW20005811:health-and-beauty---category-banner:banner-category:CI-10077697:0048219:web,banner-category,2023-12-20,2023-12-26,25,25,0
2,ww-sm:shop:browse:health-wellness:diet-sports-nutrition:protein-bars-snacks,productcategory,None,WOW20005811:health-and-beauty---category-card:content-card-category:CI-10077696:0048217:web,content-card-category,2023-12-21,2023-12-26,35,35,0
3,ww-sm:shop:browse:health-wellness:vitamins:vitamins-specials,productcategory,None,WOW20005811:health-and-beauty---category-banner:banner-category:CI-10077697:0048219:web,banner-category,2023-12-21,2023-12-26,4,4,0
4,ww-sm:shop:browse:beauty-personal-care:men-s-care:men-s-shave-gels-foams,productcategory,None,WOW20005811:health-and-beauty---category-banner:banner-category:CI-10077697:0048219:web,banner-category,2023-12-22,2023-12-26,9,9,0
5,ww-sm:shop:browse:health-wellness:everyday-market:everyday-market-health-wellness,productcategory,None,WOW20005811:health-and-beauty---category-banner:banner-category:CI-10077697:0048219:web,banner-category,2023-12-21,2023-12-24,2,2,0
6,ww-sm:shop:browse:beauty-personal-care:hair-care:colouring,productcategory,None,WOW20005811:health-and-beauty---category-card:content-card-category:CI-10077696:0048217:web,content-card-category,2023-12-20,2023-12-26,70,70,0
7,ww-sm:shop:browse:health-wellness:first-aid-medicinal:cotton-wool-cotton-buds,productcategory,None,WOW20005811:health-and-beauty---category-banner:banner-category:CI-10077697:0048219:web,banner-category,2023-12-23,2023-12-23,1,1,0
8,ww-sm:shop:browse:beauty-personal-care:hampers-gifting,productcategory,None,WOW20005811:health-and-beauty---category-banner:banner-category:CI-10077697:0048219:web,banner-category,2023-12-21,2023-12-26,20,20,0
9,ww-sm:shop:browse:beauty-personal-care:skin-care:hand-moisturiser,productcategory,None,WOW20005811:health-and-beauty---category-card:content-card-category:CI-10077696:0048217:web,content-card-category,2023-12-21,2023-12-26,6,6,0


In [172]:
check = client.query(
"""

SELECT  
    udo_page_name,
    udo_page_type,
    udo_app_section,
    udo_acs_stitch_id,
    CASE 
        WHEN udo_tealium_datasource IN ('kmm8nj', '7n58ym') THEN 'App'
        WHEN udo_tealium_datasource = 'qurl3t' THEN 'Web'
    END AS platform,
    SPLIT(udo_acs_stitch_id, ":")[OFFSET(2)] AS tealium_media_type,
    MIN(DATE(post_time, "Australia/Sydney")) AS earliest_visit,
    MAX(DATE(post_time, "Australia/Sydney")) AS latest_visit,
    COUNT(DISTINCT visitor_id) AS n_visitors,
    COUNT(DISTINCT CASE 
        WHEN SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(0)] = dim_campaign.booking_number 
        AND SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(3)] = dim_campaign.line_name 
        THEN visitor_id 
    END) AS n_visitors_exposed,
    COUNT(DISTINCT CASE 
        WHEN SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(0)] <> dim_campaign.booking_number 
        AND SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(3)] <> dim_campaign.line_name 
        THEN visitor_id 
    END) AS n_visitors_not_exposed
FROM `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_events` all_events
LEFT JOIN `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns` dim_campaign 
    ON TRIM(SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(0)]) = TRIM(dim_campaign.booking_number)
    AND TRIM(SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(3)]) = TRIM(dim_campaign.line_name)
INNER JOIN `gcp-wow-cart-data-dev-d4d7.davide.multishare_pack_bookings` multishare 
    ON multishare.string_field_0 = TRIM(SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(0)])
WHERE 
    1 = 1 
    AND DATE(post_time, "Australia/Sydney") >= DATE("2024-02-01")
    AND DATE(post_time, "Australia/Sydney") < DATE("2024-06-01")
    AND ARRAY_LENGTH(SPLIT(udo_acs_stitch_id, ":")) >= 4
    AND udo_acs_stitch_id IS NOT NULL
    AND udo_tealium_datasource IN ("qurl3t", 'kmm8nj', '7n58ym')
GROUP BY 1,2,3,4,5
ORDER BY 8 DESC
LIMIT 500
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,udo_page_name,udo_page_type,udo_app_section,udo_acs_stitch_id,platform,tealium_media_type,earliest_visit,latest_visit,n_visitors,n_visitors_exposed,n_visitors_not_exposed
0,ww-sm:shop:browse:bakery:packaged-bread-bakery,productcategory,None,WOW20011254:new-families---customer-segments-banner:banner-customer-segments:CI-10134915:0006698:web,Web,banner-customer-segments,2024-05-29,2024-05-31,12,12,0
1,ww-sm:shop:browse:bakery:bakery-specials,productcategory,None,WOW20009197:skin-care-and-cosmetics---customer-segments-banner:banner-customer-segments:CI-10149838:0007014:web,Web,banner-customer-segments,2024-05-31,2024-05-31,14,14,0
2,ww-sm:shop:search:products,search,None,WOW20011999:lunchbox---customer-segments-banner:banner-customer-segments:CI-10141788:0005699:web,Web,banner-customer-segments,2024-05-22,2024-05-31,4760,4760,0
3,ww-sm:shop:browse:fruit-veg:salad,productcategory,None,WOW20009771:wow-reach-all---leaderboard:search-display-banner:CI-10124570:0004840:web,Web,search-display-banner,2024-05-21,2024-05-31,6,6,0
4,ww-sm:shop:browse:pantry:baking:cake-dessert-mix,productcategory,None,WOW20007371:wow-reach-grocery---leaderboard:search-display-banner:CI-10123617:0006546:web,Web,search-display-banner,2024-05-29,2024-05-31,39,39,0
5,ww-sm:shop:savedlists:30681654,lists,None,WOW20011254:baby-needs---customer-segments-banner:banner-customer-segments:CI-10134479:0006698:web,Web,banner-customer-segments,2024-05-31,2024-05-31,1,1,0
6,ww-sm:shop:browse:beauty-personal-care:hair-care:hair-treatments,productcategory,None,WOW20010864:category-web-banner---hair-styling---colour:banner-category:CI-10133458:0006737:web,Web,banner-category,2024-05-29,2024-05-31,151,151,0
7,ww-sm:shop:browse:sports-fitness-outdoor-activities,productcategory,None,WOW20011254:baby-food-and-formula---customer-segments-category-card:content-card-customer-segments:CI-10134476:0006702:web,Web,content-card-customer-segments,2024-05-30,2024-05-31,2,2,0
8,ww-sm:shop:savedlists:27479674,lists,None,WOW20012639:biscuits-sweet---customer-segments-banner:banner-customer-segments:CI-10149099:0000000:web,Web,banner-customer-segments,2024-05-31,2024-05-31,1,1,0
9,ww-sm:shop:browse:pantry:baking:yeast-baking-ingredients,productcategory,None,WOW20012639:biscuits-sweet---customer-segments-category-card:content-card-customer-segments:CI-10149098:0006950:web,Web,content-card-customer-segments,2024-05-31,2024-05-31,2,2,0


In [190]:
check = client.query(
"""
SELECT 

/*DISTINCT 
    booking_number, 
    campaign_start_date,
    campaign_end_date,
    media_start_date,
    media_end_date,     
    line_name, 
    media, 
    media_type, 
*/
    --dim_campaign.* 

    media, media_type, MIN(campaign_start_date) AS min_date, MAX(campaign_end_date) AS max_date, COUNT(DISTINCT CONCAT(booking_number, line_name)) AS n_assets 
FROM `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns` dim_campaign 
INNER JOIN `gcp-wow-cart-data-dev-d4d7.davide.multishare_pack_bookings` multishare 
    ON multishare.string_field_0 = dim_campaign.booking_number --TRIM(SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(0)])
WHERE media_location = "Online"
AND booking_number IS NOT NULL 
AND campaign_end_date < CURRENT_DATE()
AND campaign_start_date >= DATE("2024-03-01")
GROUP BY 1,2
ORDER BY 4 DESC
--ORDER BY booking_number, campaign_start_date, media_start_date

"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,media,media_type,min_date,max_date,n_assets
0,WOW Reach Grocery - Content Card,Display Reach,2024-03-06,2024-08-20,67
1,WOW Reach Grocery - Leaderboard,Display Reach,2024-03-06,2024-08-20,67
2,WOW Reach Grocery - App Banners,Display Reach,2024-03-06,2024-08-20,67
3,Search App Banner 2 - Deli Meat,Results Page Display,2024-03-06,2024-08-20,22
4,Category Web Content Card - Deli Meat,Results Page Display,2024-03-06,2024-08-20,22
...,...,...,...,...,...
973,Search Web Content Card - International - Indian,Results Page Display,2024-03-13,2024-03-26,1
974,Category Web Banner - Lamb,Results Page Display,2024-03-13,2024-03-19,1
975,Category App Banner 1 - Lamb,Results Page Display,2024-03-13,2024-03-19,1
976,Search App Banner 1 - Lamb,Results Page Display,2024-03-13,2024-03-19,1


In [183]:
check = client.query(
"""
SELECT 

/*DISTINCT 
    booking_number, 
    campaign_start_date,
    campaign_end_date,
    media_start_date,
    media_end_date,     
    line_name, 
    media, 
    media_type, 
*/
    --dim_campaign.* 
    DISTINCT 

        booking_number, media_type, line_name
FROM `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns` dim_campaign 
INNER JOIN `gcp-wow-cart-data-dev-d4d7.davide.multishare_pack_bookings` multishare 
    ON multishare.string_field_0 = dim_campaign.booking_number --TRIM(SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(0)])
WHERE media_location = "Online"
ORDER BY 1,2,3
--ORDER BY booking_number, campaign_start_date, media_start_date

"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,booking_number,media_type,line_name
0,WOW20003368,Results Page Display,CI-10113137
1,WOW20003368,Results Page Display,CI-10113138
2,WOW20003368,Results Page Display,CI-10113139
3,WOW20003368,Results Page Display,CI-10113140
4,WOW20003368,Results Page Display,CI-10113141
...,...,...,...
22750,WOW20016847,Results Page Display,CI-10210740
22751,WOW20016847,Results Page Display,CI-10210741
22752,WOW20016847,Results Page Display,CI-10210742
22753,WOW20016847,Results Page Display,CI-10210743


In [5]:
check = client.query(
"""
SELECT * FROM  `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns`  LIMIT 10
"""
).result()
check_df = check.to_dataframe()
check_df 

,booking_number,campaign_start_date,campaign_end_date,line_name,media_type,individual_product_string,media_start_date,media_end_date,total_media_spend
0,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,446453,2023-10-11,2023-10-17,0.0
1,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,446476,2023-10-11,2023-10-17,0.0
2,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,446479,2023-10-11,2023-10-17,0.0
3,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,579716,2023-10-11,2023-10-17,0.0
4,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,446487,2023-10-11,2023-10-17,0.0
5,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,579736,2023-10-11,2023-10-17,0.0
6,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,257713,2023-10-11,2023-10-17,0.0
7,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,479087,2023-10-11,2023-10-17,0.0
8,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,446483,2023-10-11,2023-10-17,0.0
9,WOW20000505,2023-05-17,2023-11-21,CI-10072372,Display Search,446478,2023-10-11,2023-10-17,0.0


In [12]:
check = client.query(
"""


CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns`  AS (
    with step_one AS (
        SELECT
            booking_number,
            campaign_start_date,
            campaign_end_date,
            line_name,
            CASE
                WHEN media_location LIKE ('%TV%') THEN 'Mass_Social_Media'
                WHEN media_type IN (
                    'Counter Card',
                    'POC',
                    'Spotify',
                    'YouTube',
                    'Social',
                    'POC',
                    'Social - Wrapper',
                    'Build Capacity'
                ) THEN 'Mass_Social_Media'
                WHEN media_location IN ('Category', 'Aisle', 'Store', 'Screens', 'Emma Reach') THEN 'in-store'
                WHEN media LIKE ('%FB Post%') THEN 'Mass_Social_Media'
                WHEN media_type = 'Always On eDMs' THEN 'eDM'
                WHEN media_type = 'Single Send eDM' THEN 'eDM'
                WHEN media_type = 'BRANDED SHOP' THEN 'Branded Shop'
                WHEN media_type = 'CONTENT CARD' THEN 'Content Card'
                WHEN media_type = 'Digital Standalone Catalogue' THEN 'Catalogue'
                WHEN media_type = 'Display Recipes - Wrapper' THEN 'Display Recipes'
                WHEN media_type = 'RECIPE PACKAGE' THEN 'Display Recipes'
                WHEN media_type = 'Recipe Package' THEN 'Display Recipes'
                WHEN media_type = 'Standalone Catalogue - Double Spot' THEN 'Catalogue'
                WHEN media_type = 'Standalone Catalogue - Single Spot' THEN 'Catalogue'
                WHEN media_type = 'Standalone Catalogue- Third Page Vertical' THEN 'Catalogue'
                WHEN media_type = 'Catalogue Card' THEN 'Catalogue'
                WHEN media_type = 'BF Meal' THEN 'Others'
                WHEN media_type = 'Sampling' THEN 'Others'
                WHEN media_type = 'Bus Stop' THEN 'in-store'
                WHEN media_type = 'Display Promo Tile' THEN 'in-store'
                WHEN media_type = 'Aisle Fin' THEN 'in-store'
                WHEN media_type = 'Additional Production Fees' THEN 'Others'
                WHEN media_type = 'Event' THEN 'Others'
                WHEN media_type = 'National Radio' THEN 'Mass_Social_Media'
                WHEN media_type = 'Chilled Bus Stop & Decal Package' THEN 'in-store'
                WHEN media_type = 'National TV' THEN 'Mass_Social_Media'
                WHEN media_type = 'Press' THEN 'Mass_Social_Media'
                WHEN media_type = 'Content Card' THEN 'Others'
                WHEN media_type = 'Competition Management Fee' THEN 'Others'
                WHEN media_type = 'Prize pool' THEN 'Others'
                WHEN media_type = 'Chilled Bus Stop & Decal Package' THEN 'in-store'
                WHEN media_type = 'Chilled Fin Package' THEN 'in-store'
                WHEN media_type = 'Editorial' THEN 'Mass_Social_Media'
                WHEN media_type = 'BUS STOP' THEN 'in-store'
                WHEN media_type = 'Door Take' THEN 'in-store'
                WHEN media_type = 'Freezer Package' THEN 'in-store'
                WHEN media_type = 'PL - Mag' THEN 'Others'
                WHEN media_type = 'PL - Fresh Mag' THEN 'Others'
                WHEN media_type = 'PL - Demos' THEN 'Others'
                WHEN media_type = 'Pelmet' THEN 'in-store'
                WHEN media_type = 'Overlay Card' THEN 'in-store'
                WHEN media_type = 'Mass/Social Media' THEN 'Others'
                WHEN lower(media_type) like "%catalogue%" THEN 'Catalogue'
                WHEN LOWER(media_type) LIKE "%homepage%" THEN "Home Page"
                ELSE
                media_type
            END
            AS media_type,
            individual_product_string,
            MIN(media_start_date) AS media_start_date,
            MAX(media_end_date) AS media_end_date,
            SUM(quantity) * SUM(media_spend) AS total_media_spend

        FROM `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns`,
        UNNEST(SPLIT(quoteline_sku, ',')) AS individual_product_string

        WHERE
            
            business_unit = 'Supermarket'
            AND booking_number IS NOT NULL
            AND quote_status NOT IN ('Not Approved','Draft','For Review','Sent')
            AND media_location NOT IN ('Category','Aisle','Store','Screens','Emma Reach')
            AND media_location NOT LIKE ('%TV%')
            AND media_type NOT IN ('Counter Card','POC','Spotify','YouTube','Social','POC','Social - Wrapper')
            AND (media_type <> 'Event')
            AND (media_location <> 'None' OR media LIKE ('%FB Post%') OR media LIKE ('%Branded%') OR media LIKE ('%Segments%'))
            AND media NOT IN ('Competition Management Fee','Additional Production Fees')
        AND media_start_date >= DATE("2023-10-01")
        AND media_end_date < CURRENT_DATE("Australia/Sydney")
        GROUP BY
            1,2,3,4,5,6
    ) 

    SELECT * 
    FROM step_one
    WHERE
        media_type NOT IN ('in-store','Others','eDM')
        AND media_type NOT LIKE ('%Woolworths Rewards%')
        AND media_type NOT LIKE ('%SAMPLING%') 
        AND media_type NOT LIKE "%Road Block%"
        AND media_type <> 'Radio'
        AND media_type <> 'Mass_Social_Media'
        AND LOWER(media_type) NOT LIKE "%branded shop%"

);
"""
).result()
check_df = check.to_dataframe()
check_df 

""


In [16]:
check = client.query(
"""
SELECT * FROM `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns` LIMIT 4
"""
).result()
check_df = check.to_dataframe()
check_df 

,quote_name,booking_number,business_unit,quote_status,quote_sku,net_amount,opportunity_owner,account,line_name,media,media_location,media_type,media_start_date,media_end_date,campaign_start_date,campaign_end_date,original_campaign_start_date,original_campaign_end_date,campaign_duration_weeks,store_list,quoteline_sku,revenue_sku,quantity,media_spend,duration_weeks,opportunity_name,campaign_objective,event_name,placement,online_department,brand,sku_item_name,quoteline_category,commercial_category,cnz_category_patch,line_name_all,original_booking_number,load_dttm,compliance_media_type
0,WOW20016829,WOW20016829,Supermarket,Stage 100%,None,22500.0,Hamid Alfughi,PZ Cussons Family Care (HHP) Deodorant (HBB),CI-10209521,Online Sampling,Online,Sampling,2024-11-06,2024-11-12,2024-11-06,2024-11-12,2024-11-06,2024-11-12,1,None,905268,905268,1.0,22500.0,1,Radiant -Online Sampling - $22.5k,Acquire New Customers,None,None,None,Radiant,None,None,None,None,"CI-10209521,",None,2024-08-12 10:20:57.219148,None
1,Q-00495,WOW10000496,Supermarket,Not Approved,None,40000.0,Hayley Clarke,Beam Suntory (DB),QL-0001009,eDM Themed,Orange Reports,Woolworths Rewards,NaT,NaT,NaT,NaT,NaT,NaT,<NA>,None,None,None,200000.0,0.0,<NA>,Nutriboost w2,None,None,None,None,None,None,None,None,None,QL-0001009,None,2024-08-12 10:20:57.219148,None
2,Q-00495,WOW10000496,Supermarket,Not Approved,None,40000.0,Hayley Clarke,Beam Suntory (DB),QL-0001011,eDM Band2 - Themed,Orange Reports,Woolworths Rewards,NaT,NaT,NaT,NaT,NaT,NaT,<NA>,None,None,None,1.0,40000.0,<NA>,Nutriboost w2,None,None,None,None,None,None,None,None,None,QL-0001011,None,2024-08-12 10:20:57.219148,None
3,Q-01160,WOW10001161,Supermarket,Not Approved,None,0.2,Hayley Clarke,Haleon Australia Pty Ltd,QL-0003369,eDM Always On,Orange Reports,Woolworths Rewards,NaT,NaT,NaT,NaT,NaT,NaT,<NA>,None,None,None,1.0,0.2,<NA>,GSK annual plan,Grow Existing Customers,None,None,None,None,None,None,None,None,QL-0003369,None,2024-08-12 10:20:57.219148,None


In [166]:
check = client.query(
"""
        with step_one AS (
            SELECT DISTINCT 
                booking_number,
                media_type,
                CASE WHEN media_type <> "Display Aisle and Category" THEN 1 ELSE 0 END AS non_display_aisle_asset
            FROM `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` 
            ORDER BY booking_number
        ) 
        
        SELECT DISTINCT cc.booking_number, cc.campaign_start_date, cc.campaign_end_date --cc.* EXCEPT(individual_product_string), ARRAY_AGG(DISTINCT individual_product_string IGNORE NULLS ORDER BY individual_product_string) AS skus
        FROM `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` cc 
        LEFT JOIN (SELECT DISTINCT booking_number FROM step_one WHERE non_display_aisle_asset = 1) non
            ON non.booking_number = cc.booking_number 
        --INNER JOIN `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_nine_final_results` final 
        --    ON final.booking_id = cc.booking_number
        WHERE cc.booking_number = "WOW20005261"        
        
"""
).result()
check_df = check.to_dataframe()
check_df 

,booking_number,campaign_start_date,campaign_end_date
0,WOW20005261,2023-09-27,2024-01-09
